# Adding Baselines to the already existant REINFORCE Algorithm

\begin{array}
A_t &= G_t - V(s_t) \\
\nabla J(\theta) &= \sum_{t=0}^{T-1} \nabla 
\log \pi(a_t/s_t) \times A_t
\end{array}


In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
from torch.autograd import Variable
env = gym.make('CartPole-v1')

# Let's create a custom environment
https://www.youtube.com/watch?v=bD6V3rcr_54 is the reference for this.An RL Environment to adjust the temperature of your shower

In [3]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
import random

In [6]:
# creating placeholder functions
class ShowerEnv(Env):
    def __init__(self):
        # defining the actions
        self.action_space = Discrete(3) # 3 discrete actions - meddling with the shower handle
        # Temperature Array
        self.observation_space = Box(low=np.array([0]),high=np.array([100]))
        # Set start temperature
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self,action):
        self.state += action -1
        self.shower_length -= 1
        
        if self.state >=37 and self.state <=38:
            reward = 1
        else:
            reward = -1
        if self.shower_length<= 0:
            truncated = True
            terminated = True
        else:
            truncated = False
            terminated = False
        self.state += random.randint(-1,1)
        info = {}
        return self.state, reward, terminated, truncated, info
    def render(self):
        None
    def reset(self):
        self.state = 38+ random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        info = {}
        return self.state, info 

In [8]:
env = ShowerEnv()

In [13]:
n_episode = 100

for episode in range(n_episode):

    action = env.action_space.sample()
    new_state, reward, terminated, truncated,info = env.step(action)
    print(new_state, reward, terminated, truncated,info )

38 1 True True {}
38 1 True True {}
39 1 True True {}
41 -1 True True {}
40 -1 True True {}
40 -1 True True {}
40 -1 True True {}
41 -1 True True {}
41 -1 True True {}
42 -1 True True {}
43 -1 True True {}
41 -1 True True {}
40 -1 True True {}
41 -1 True True {}
41 -1 True True {}
40 -1 True True {}
40 -1 True True {}
40 -1 True True {}
42 -1 True True {}
40 -1 True True {}
40 -1 True True {}
41 -1 True True {}
42 -1 True True {}
44 -1 True True {}
46 -1 True True {}
47 -1 True True {}
47 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
44 -1 True True {}
45 -1 True True {}
45 -1 True True {}
47 -1 True True {}
46 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
44 -1 True True {}
42 -1 True True {}
43 -1 True True {}
44 -1 True True {}
45 -1 True True {}
47 -1 True True {}
45 -1 True True {}
45 -1 True True {}
44 -1 True True {}
45 -1 True True {}
43 -1 True True {}
43 -1 True True {}
42 -1 True True {}
40 -1 True True {}
40 -1 True True